In [23]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable 
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from torchvision import datasets, transforms
# from PIL import Image

In [24]:
BATCH_SIZE=32 #大概需要2G的显存
EPOCHS=5
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [25]:
def left_top(x):
    nsam = x.shape[0]
    nrow = x.shape[1]
    ncol = x.shape[2]
    z = np.zeros((nsam, nrow, ncol), dtype=np.double)
    data1 = np.concatenate((x, z), axis=1)
    data2 = np.concatenate((z, z), axis=1)
    data3 = np.concatenate((data1, data2), 2)
    return data3
def right_bottom(x):
    nsam = x.shape[0]
    nrow = x.shape[1]
    ncol = x.shape[2]
    z = np.zeros((nsam, nrow, ncol), dtype=np.double)
    data1 = np.concatenate((z, z), axis=1)
    data2 = np.concatenate((z, x), axis=1)
    data3 = np.concatenate((data1, data2), 2)
    return data3

In [26]:
def getType(dataset,x):
    index = np.where(dataset[:, 0] == x)
    for i in index:
        Type = dataset[i]
    return Type

In [27]:
def getPixel(x):
    pixel = x[:, 1:]
    return pixel
def getLabel(x):
    label = x[:, 0]
    return label

In [28]:
class train(Dataset):
    def __init__(self):
        
        mnist_train = pd.read_csv('mnist_train.csv')
        train = mnist_train.values

        mnist_train_0=getType(train,0)
        mnist_train_1=getType(train,1)   
        mnist_train_2=getType(train,2)
        mnist_train_3=getType(train,3)   
        
            
        nrow = 28
        ncol = 28

        # X and label
        x1 = getPixel(mnist_train_1)
        x0 =getPixel(mnist_train_0)
        y1 = getLabel(mnist_train_1)
        y0 = getLabel(mnist_train_0)
        x2 = getPixel(mnist_train_2)
        x3 =getPixel(mnist_train_3)
        y2= getLabel(mnist_train_2)
        y3 = getLabel(mnist_train_3)

        # X reshape to 28 28
               
        nx1 = x1.shape[0]
        nx0 = x0.shape[0]
        nx2 = x2.shape[0]
        nx3 = x3.shape[0]
       
        x0_reshape = x0.reshape((nx0, nrow, ncol))
        x1_reshape = x1.reshape((nx1, nrow, ncol))
        x2_reshape = x2.reshape((nx2, nrow, ncol))
        x3_reshape = x3.reshape((nx3, nrow, ncol))


        left0 = left_top(x0_reshape)
        right1 = right_bottom(x1_reshape)
        
        left2=left_top(x2_reshape)
        right3=right_bottom(x3_reshape)
        
        
        
        train_data1 = np.concatenate((left0, right1), axis=0)
        train_data2 = np.concatenate((left2, right3), axis=0)
        train_data= np.concatenate((train_data1, train_data2), axis=0)
        
        nn = train_data.shape[0]

        # reshape to 3136 which is 56*56
        train_data_reshape = train_data
        train_data_reshape = torch.from_numpy(train_data_reshape)
        train_data_reshape = torch.unsqueeze(train_data_reshape, 1)
        
        yy1 = np.concatenate((y0, y1), axis=0)
        yy2=np.concatenate((y2, y3), axis=0)
        yy=np.concatenate((yy1, yy2), axis=0)
        
        yy=torch.from_numpy(yy)
      
        self.len = yy.shape[0]

        self.x_data = train_data_reshape

        self.y_data = yy


    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
      

    def __len__(self):
        return self.len
dataset1= train()
train_loader = DataLoader(dataset=dataset1,
                          batch_size=32,
                          shuffle=True,
                          num_workers=2)

In [29]:
class test(Dataset):
    def __init__(self):
  
        mnist_test = pd.read_csv('mnist_test.csv')
        testtest= mnist_test.values
        
        
        test0=getType(testtest,0)
        test1=getType(testtest,1)
        test2=getType(testtest,2)
        test3=getType(testtest,3)
        nrow = 28
        ncol = 28

        # X and label
        x1 = getPixel(test1)
        x0 =getPixel(test0)
        y1 = getLabel(test1)
        y0 = getLabel(test0)
        x2 = getPixel(test2)
        x3 =getPixel(test3)
        y2 = getLabel(test2)
        y3 = getLabel(test3)



        # X reshape to 28 28
        nx1 = x1.shape[0]
        nx0 = x0.shape[0]
        nx2 = x2.shape[0]
        nx3 = x3.shape[0]
      
        x0_reshape = x0.reshape((nx0, nrow, ncol))
        x1_reshape = x1.reshape((nx1, nrow, ncol))
        x2_reshape = x2.reshape((nx2, nrow, ncol))
        x3_reshape = x3.reshape((nx3, nrow, ncol))


        left0 = left_top(x0_reshape)
        right1 = right_bottom(x1_reshape)
        
        left2=left_top(x2_reshape)
        right3=right_bottom(x3_reshape)
        
        
        
        train_data1 = np.concatenate((left0, right1), axis=0)
        train_data2 = np.concatenate((left2, right3), axis=0)
        train_data= np.concatenate((train_data1, train_data2), axis=0)

        

        nn = train_data.shape[0]

        
        train_data_reshape = train_data
        train_data_reshape = torch.from_numpy(train_data_reshape)
        train_data_reshape = torch.unsqueeze(train_data_reshape, 1)
        
        yy1 = np.concatenate((y0, y1), axis=0)
        yy2=np.concatenate((y2, y3), axis=0)
        yy=np.concatenate((yy1, yy2), axis=0)
        
        yy=torch.from_numpy(yy)
      
        self.len = yy.shape[0]

        self.x_data = train_data_reshape

        self.y_data = yy


    def __getitem__(self, index):
        # img = Image.fromarray(self.x_data[index])
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len
dataset2 = test()
test_loader = DataLoader(dataset=dataset2,
                          batch_size=32,
                          shuffle=True,
                          num_workers=2)

In [30]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        # 1,28x28
        self.conv1=nn.Conv2d(1,10,5) # 10, 24x24
        self.conv2=nn.Conv2d(10,20,3) # 128, 10x10
        self.fc1 = nn.Linear(2880,300)
        self.fc2 = nn.Linear(300,4)
    def forward(self,x):
        in_size = x.size(0)
        out = self.conv1(x) #24
        out = F.relu(out)
        out = F.max_pool2d(out, 2, 2)  #12
        out = self.conv2(out) #10
        out = F.relu(out)
        out = F.max_pool2d(out, 2, 2)  #12
        out = F.relu(out)   
        out = out.view(in_size,-1)
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        # out = F.log_softmax(out,dim=1)
        # out = F.softmax(out,dim=1)
        return out

In [31]:
model = ConvNet()
model = model.to(DEVICE)
model= model.double()
optimizer = optim.Adam(model.parameters())
# criterion = nn.CrossEntropyLoss()

In [32]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        # print(data.size(), target.size())
        optimizer.zero_grad()
        output=model(data)
        # print(output)
        #print(target)
        # loss = F.nll_loss(output, target)
        # print(loss)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if(batch_idx+1)%100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data.double())
            print(output)
            print(target)
            test_loss += F.cross_entropy(output, target, reduction='sum').item() # 将一批的损失相加
            pred = output.max(1, keepdim=True)[1] # 找到概率最大的下标
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))






In [33]:
for epoch in range(1, EPOCHS + 1):
    train(model, DEVICE, train_loader, optimizer, epoch)
    test(model, DEVICE, test_loader)

Train Epoch: 1 [3168/24754 (13%)]	Loss: 0.021714
Train Epoch: 1 [6368/24754 (26%)]	Loss: 0.025134
Train Epoch: 1 [9568/24754 (39%)]	Loss: 0.000024
Train Epoch: 1 [12768/24754 (52%)]	Loss: 0.000808
Train Epoch: 1 [15968/24754 (64%)]	Loss: 0.000047
Train Epoch: 1 [19168/24754 (77%)]	Loss: 0.000172
Train Epoch: 1 [22368/24754 (90%)]	Loss: 0.000075
tensor([[ 22.1139, -22.3416,   9.0875,  -3.1564],
        [ 25.4190, -24.9409,   9.1895,  -3.7588],
        [ -3.1863,  15.3647,  -7.8207,  -6.1470],
        [ -5.1176,  20.9034, -10.9093, -11.5429],
        [  5.4475, -16.8374,  17.1707,  -3.1084],
        [ -3.7658,  -1.8075, -11.6622,  13.2228],
        [  3.7754, -17.7166,  19.2881,  -1.8761],
        [ -3.9152,  17.1528,  -9.8871,  -8.7443],
        [ 20.5529, -20.8222,   6.8376,  -3.0793],
        [ 18.4664, -23.1329,  11.4993,  -2.9734],
        [  7.5239, -19.0526,  19.4399,  -2.6597],
        [ -4.2275,  16.4318, -10.5813,  -7.9034],
        [  5.6987, -18.7530,  20.1028,  -2.2700],
   

tensor([[ 23.2867, -26.3033,   9.6432,  -3.4413],
        [ -7.4476,  -0.4325, -16.1747,  17.2960],
        [ 22.3138, -22.7094,   8.1135,  -2.8538],
        [  1.6732, -22.9356,  26.9623,  -2.5629],
        [ -3.1470, -21.0949,  26.7916,  -2.7542],
        [ -6.9100,  -2.3334, -17.2600,  19.7517],
        [ 21.1016, -23.5460,  10.3491,  -3.7033],
        [  0.9794, -19.9717,  23.9112,  -3.2910],
        [ 24.4507, -24.7670,   7.4408,  -2.9253],
        [ -0.7932, -21.0851,  24.9003,  -3.0082],
        [ 20.0973, -21.8724,   9.2133,  -3.2047],
        [ -6.5730,  -0.4999, -13.6726,  16.1668],
        [  7.0805, -20.4819,  20.8756,  -2.8679],
        [ -5.0339,  15.3570, -11.7374,  -5.1439],
        [ -6.3039,  -0.4380, -15.4538,  16.7056],
        [ 24.1374, -25.1157,  10.7182,  -3.8104],
        [ 15.9717, -19.6113,  10.5192,  -3.9649],
        [ -4.6782,  15.3461, -11.6323,  -5.8108],
        [ 23.0203, -25.0873,   8.0429,  -4.0322],
        [ -0.4879, -21.1447,  25.8779,  -3.1189],


tensor([[ -2.0673,   7.6088,  -5.7415,  -1.4303],
        [ 23.4709, -23.0545,   9.8558,  -3.6881],
        [ 24.0450, -26.1481,   9.1402,  -4.4052],
        [ 20.0765, -22.5289,  10.5987,  -2.5877],
        [  1.7040, -15.7591,  19.5804,  -2.7144],
        [ -5.5192,  -0.6412, -14.1640,  14.0557],
        [ 21.2281, -22.3572,   9.5029,  -3.1562],
        [ -4.4260,  19.0976, -10.1307,  -8.8050],
        [ -0.3309, -21.2490,  24.8228,  -4.3503],
        [ 22.3135, -23.8342,   8.4767,  -3.3243],
        [  2.2263, -14.8088,  17.5302,  -3.4967],
        [ 21.2430, -24.1518,  10.3773,  -3.3263],
        [ -2.1605, -18.0662,  22.7240,  -3.3929],
        [ -4.6764,  14.4194, -10.2694,  -4.7932],
        [ -5.1974,   2.2674, -13.2221,  10.6940],
        [ 23.9771, -22.3518,   9.4715,  -3.4608],
        [ 23.8972, -23.7173,   9.2007,  -4.1873],
        [  4.3195, -17.1531,  18.3830,  -3.0125],
        [  2.0444, -15.9280,  19.6315,  -1.6089],
        [ -6.1360,   0.0534, -15.3393,  16.0782],


tensor([[ -4.0195,   9.7960,  -8.2129,  -2.7024],
        [ -6.4035,   2.0428, -13.2046,  10.8315],
        [ -6.6414,  -0.6730, -14.7226,  15.7805],
        [  3.0223, -20.4927,  24.6684,  -4.9725],
        [  1.8280, -18.9146,  21.1924,  -2.1294],
        [ 23.2287, -23.6771,  10.9502,  -2.9283],
        [ 19.4021, -22.3038,   9.1995,  -3.2250],
        [ -5.7571,  -0.1473, -13.7263,  13.4755],
        [ 20.9408, -26.0289,  13.7724,  -3.2185],
        [ -3.7183,  18.5428, -10.1572,  -7.4595],
        [ -3.5851,  14.8663,  -9.5895,  -5.5601],
        [ -0.4902, -18.0276,  22.2331,  -2.0930],
        [ -3.7394,  17.8988, -10.7617,  -8.1174],
        [ -3.7352,  16.1319,  -9.8971,  -7.5009],
        [ -4.0871,  16.9483, -11.2106,  -6.1255],
        [ -1.0076, -22.1264,  27.0387,  -2.1253],
        [ -4.7605,   3.6206, -13.8290,   8.8462],
        [  0.2909, -18.0586,  21.8367,  -4.0063],
        [ 20.5675, -21.4254,   9.6609,  -3.3836],
        [ -4.4498,  -2.8344, -13.9832,  18.0171],


tensor([[-7.2733e+00,  1.0882e+00, -1.8504e+01,  1.8330e+01],
        [-5.7441e+00,  3.0023e+00, -1.3661e+01,  9.8669e+00],
        [ 9.5197e-01, -2.0040e+01,  2.4320e+01, -2.9046e+00],
        [ 1.9761e+01, -2.0883e+01,  1.0969e+01, -3.5505e+00],
        [-6.9296e+00,  8.3003e-01, -1.5952e+01,  1.7027e+01],
        [-4.6475e+00,  1.5071e+01, -9.6755e+00, -5.7652e+00],
        [ 1.3840e+01, -1.7886e+01,  1.0306e+01, -3.3477e+00],
        [ 2.1976e+01, -2.4842e+01,  9.0302e+00, -3.9243e+00],
        [ 1.2519e+00, -1.1374e+01,  1.3737e+01, -1.5004e+00],
        [ 3.9247e+00, -2.2253e+01,  2.6442e+01, -1.8784e+00],
        [ 1.6522e+00, -1.4645e+01,  1.7379e+01, -2.9256e+00],
        [ 3.2938e+00, -1.8402e+01,  2.1333e+01, -2.8637e+00],
        [-3.4590e+00,  1.3276e+01, -8.8757e+00, -5.1864e+00],
        [ 6.1020e+00, -1.7712e+01,  1.6701e+01, -2.5493e+00],
        [-3.5187e-02, -1.2716e+01,  1.4232e+01, -3.4139e+00],
        [-5.5268e+00,  8.1849e-01, -1.4208e+01,  1.2411e+01],
        

...

tensor([[-20.2633,  17.1402, -17.6115, -11.4882],
        [ -9.3568, -18.6562,  12.1012, -21.7710],
        [-12.5585, -22.5345,  15.9503, -23.8175],
        [ -8.3257, -17.8049,  12.2281, -19.9077],
        [-11.0258, -24.7546,  14.4433, -28.0310],
        [ -7.3448,   9.2882,  -3.6102,  -7.4417],
        [-15.9715,  18.7582, -13.8383, -13.4819],
        [-10.9069,  13.3453,  -8.4476,  -8.7994],
        [-43.9072,  -9.1132, -44.6366,  20.4807],
        [-38.0255,  -6.8918, -38.9318,  18.2952],
        [-14.7406,  17.8477,  -8.7940, -12.5615],
        [-21.0659,  -5.7000, -24.0926,  10.7906],
        [-27.4174,  -6.5364, -31.3258,  16.7348],
        [-42.0855, -10.3268, -46.8080,  21.7297],
        [ -8.7449,  11.6358,  -4.5409,  -8.0683],
        [ -6.5940, -18.1545,   9.3055, -18.2459],
        [-46.3170,  -7.9281, -47.4402,  25.5245],
        [-10.6708,  13.0302,  -7.4185,  -8.4575],
        [  9.9563,  -3.2262,  -8.9393,  -2.7656],
        [-12.0401,   4.6373,  -9.1666,  -4.4722],


tensor([[-41.2599, -10.4803, -45.8086,  28.2322],
        [ -6.2146,  -9.7629,   7.2411, -12.4083],
        [ -9.0978, -22.1831,  12.2877, -23.3650],
        [ -7.7925, -16.2832,   8.4485, -19.0438],
        [ -4.0744, -11.6163,   4.9534, -13.0035],
        [-35.4006,  -8.6953, -36.9574,  13.2102],
        [ 16.8778,  -5.0941, -14.4785,  -6.2055],
        [ -8.0964, -11.6245,   8.0398, -13.2674],
        [-11.4265, -17.2511,  13.3086, -17.6742],
        [-28.4283,  -7.7122, -32.2483,  18.1338],
        [ -8.1722,   5.7578,  -5.3443,  -3.2411],
        [  8.6635,  -6.0569,  -3.6612,  -5.0037],
        [ -0.8240,  -7.1100,   1.8987, -10.3253],
        [-37.2209,  -9.1845, -42.3942,  23.0813],
        [-22.9027,  25.4884, -18.2722, -17.1626],
        [  7.3154,  -3.0110,  -5.8215,  -2.7487],
        [  4.9534,  -5.6678,  -3.1603,  -3.4528],
        [ -8.0967, -17.0605,   9.3323, -20.6956],
        [-44.3086, -10.7460, -47.4773,  27.4457],
        [ -7.5764,   8.2015,  -4.5871,  -6.5162],


tensor([[-37.8334,  -8.6385, -38.9374,  18.3048],
        [-28.7846,  25.5024, -26.6015, -15.7595],
        [ -7.6816,   5.7962,  -4.8874,  -3.3341],
        [ -7.8607,   5.9827,  -5.4204,  -3.5741],
        [-12.5946,  15.4230,  -8.1453, -10.1877],
        [ -5.6837, -16.7322,   9.4834, -18.5930],
        [-18.9237,  13.2743, -17.5960,  -8.3273],
        [-37.4840,  -7.4955, -38.3172,  17.1767],
        [-24.3841,  18.6229, -22.0628, -12.4564],
        [-15.5317, -24.4244,  17.4626, -24.1783],
        [  5.6877,  -6.9499,  -2.1795,  -4.6629],
        [-42.6134,  -9.9164, -44.9815,  18.9889],
        [-16.1840, -31.5261,  19.1147, -32.9229],
        [-10.7963, -17.2718,  11.7712, -18.8330],
        [ 10.5049,  -8.7563,  -6.2260,  -5.2017],
        [-13.2436, -26.7858,  15.3973, -31.1236],
        [ -8.0618, -15.3602,   8.9351, -16.3251],
        [  5.9844,  -4.3616,  -4.0849,  -2.8766],
        [-12.5349,   5.2271,  -9.2715,  -4.1862],
        [-16.1723,  19.4326, -10.8562, -12.6072],


tensor([[-10.8516, -18.3661,  14.8049, -17.9159],
        [-13.8108, -25.5889,  15.9398, -24.1520],
        [-10.3237, -25.8171,  13.4379, -27.0680],
        [ -7.3569, -16.6167,   9.7674, -18.9441],
        [ -9.8082,  11.6249,  -7.7966,  -8.1775],
        [-15.4580,   6.7031, -17.5796,  -4.4317],
        [-46.0796,  -9.7158, -47.6163,  23.6814],
        [-36.4076,  -9.0844, -40.7980,  23.6217],
        [  9.4053,  -6.2502,  -6.8489,  -6.4526],
        [-26.9709,  -7.9627, -31.4381,  17.6023],
        [-22.0163,  19.9944, -20.5285, -12.8922],
        [-23.3440,  26.4623, -18.9602, -17.0293],
        [-29.5370,  -9.8788, -34.6528,  26.1728],
        [ 12.0759,  -4.5572, -10.9956,  -3.6177],
        [-17.8962,  20.3840, -13.8005, -13.2936],
        [ -9.1936,   3.3133,  -8.7347,  -2.3586],
        [ -4.4049, -15.5438,   6.9619, -17.6133],
        [-22.8703,  22.5110, -19.9588, -13.5550],
        [-45.7431,  -9.2564, -48.6134,  20.8264],
        [-13.6951,  16.0738,  -9.8292, -11.1656],


tensor([[-21.9599,  -4.9159, -26.6713,  19.9177],
        [  9.6541,  -3.5768,  -7.3047,  -3.2896],
        [-17.3823,  20.4918, -12.8398, -13.4987],
        [-24.1244,  26.2733, -20.2951, -14.9338],
        [-21.9899,  16.5500, -20.2737, -11.7997],
        [-19.8117, -37.4912,  23.1149, -37.0342],
        [-10.2854, -23.9623,  14.1671, -27.3769],
        [ 19.7874, -12.2060, -12.0883,  -8.3836],
        [-16.9097, -31.4648,  18.5982, -31.4183],
        [ 15.8318,  -9.8670, -10.2039,  -6.4842],
        [-28.2832,  -7.2649, -31.8248,  17.6516],
        [ -6.5314, -20.9763,   7.8493, -23.3290],
        [  5.4551,  -4.3219,  -3.4286,  -3.1323],
        [-36.4175,  -7.3247, -37.2582,  17.7566],
        [-41.6886, -10.0728, -47.2620,  25.8528],
        [-10.1166, -20.1449,  14.0218, -21.4108],
        [ -5.5469, -11.9666,   5.7494, -16.3929],
        [ 17.3156,  -8.7152, -13.2588,  -6.1790],
        [-25.1452,  -7.5806, -30.6735,  21.8276],
        [-14.3336,  15.7182, -10.5809, -10.3582],
